Import các thư viện cần thiết

In [92]:
import numpy as np  
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB

Đọc dữ liệu từ file "banking.csv"

In [93]:
data = pd.read_csv("D:/Code/python/mat3533/practice03/data/banking.csv")
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


Chuyển đổi các trường dữ liệu sang thuần số

In [94]:
dict_month = {'jan' : 1, 'feb' : 2, 'mar' : 3, 'apr' : 4, 'may' : 5, 'jun' : 6,
'jul' : 7, 'aug' : 8, 'sep' : 9, 'oct' : 10, 'nov' : 11, 'dec' : 12}
data['month'] = data['month'].map(dict_month)
dict_day = {'sun' : 1, 'mon' : 2, 'tue' : 3, 'wed' : 4, 'thu' : 5, 'fri' : 6,
'sat' : 7}
data['day_of_week'] = data['day_of_week'].map(dict_day)
data['default'] = data['default'].map({'yes' : 1,'no' : 0})
data['housing'] = data['housing'].map({'yes' : 1,'no' : 0})
data['loan'] = data['loan'].map({'yes' : 1,'no' : 0})

marital_dummies = pd.get_dummies(data['marital'], prefix = 'marital', dtype=int)
marital_dummies.drop('marital_unknown', axis=1, inplace=True)
data = pd.concat([data, marital_dummies], axis=1)
data.drop(['marital'], axis=1, inplace=True)

job_dummies = pd.get_dummies(data['job'], prefix = 'job', dtype=int)
job_dummies.drop('job_unknown', axis=1, inplace=True)
data= pd.concat([data, job_dummies], axis=1)
data.drop(['job'], axis=1, inplace=True)

education_dummies = pd.get_dummies(data['education'], prefix = 'education', dtype=int)
education_dummies.drop('education_unknown', axis=1, inplace=True)
data = pd.concat([data, education_dummies], axis=1)
data.drop(['education'], axis=1, inplace=True)

contact_dummies = pd.get_dummies(data['contact'], prefix = 'contact', dtype=int)
data = pd.concat([data, contact_dummies], axis=1)
data.drop(['contact'], axis=1, inplace=True)

poutcome_dummies = pd.get_dummies(data['poutcome'], prefix = 'poutcome', dtype=int)
data = pd.concat([data, poutcome_dummies], axis=1)
data.drop(['poutcome'], axis=1, inplace=True)

data['pdays'] = data['pdays'].apply(lambda row: 0 if row == -1 else 1)

data.dropna(inplace=True)

Chia dữ liệu thành phần Training và phần Test theo tỷ lệ 8:2, trong đó với phần Test, trường y sẽ không dùng đến.

In [57]:
x = data.drop('y', axis=1)
y = data['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Sử dụng mô hình hồi quy logistic với phần dữ liệu Training, và áp dụng để dự đoán phần Test. Cuối cùng sử dụng các độ đo Accuracy, precision, recall và F1-score (β = 1) để kiểm tra độ chính xác của mô hình 

In [96]:
start_time = time.time()

n_train = len(y) * 8 // 10
logistic_model = LogisticRegression(max_iter=n_train)
logistic_model.fit(x_train, y_train)
logistic_predict = logistic_model.predict(x_test)
logistic_accuracy = accuracy_score(y_test, logistic_predict)
logistic_precision = precision_score(y_test, logistic_predict, average=None)
logistic_recall = recall_score(y_test, logistic_predict, average=None)
logistic_f1 = f1_score(y_test, logistic_predict, average=None)

end_time = time.time()
logistic_time = end_time - start_time

In ra kết quả khi sử dụng mô hình hồi quy logistic

In [97]:
print("Logistic Regression:")
print(f"Accuracy: {logistic_accuracy}")
print(f"Precision: {logistic_precision}")
print(f"Recall: {logistic_recall}")
print(f"F1-score: {logistic_f1}\n")

Logistic Regression:
Accuracy: 0.8988375746151429
Precision: [0.91663831 0.68442623]
Recall: [0.9722122  0.40533981]
F1-score: [0.94360771 0.50914634]



Sử dụng mô hình hồi quy Naive Bayes với phần dữ liệu Training, và áp dụng để dự đoán phần Test. Cuối cùng sử dụng các độ đo Accuracy, precision, recall và F1-score (β = 1) để kiểm tra độ chính xác của mô hình 

In [98]:
start_time = time.time()

nb_model = GaussianNB()
nb_model.fit(x_train, y_train)
nb_predict = nb_model.predict(x_test)
nb_accuracy = accuracy_score(y_test, nb_predict)
nb_precision = precision_score(y_test, nb_predict, average=None)
nb_recall = recall_score(y_test, nb_predict, average=None)
nb_f1 = f1_score(y_test, nb_predict, average=None)

end_time = time.time()
nb_time = end_time - start_time

In ra kết quả khi sử dụng mô hình Naive Bayes

In [99]:
print("Naive Bayes:")
print(f"Accuracy: {nb_accuracy}")
print(f"Precision: {nb_precision}")
print(f"Recall: {nb_recall}")
print(f"F1-score: {nb_f1}")

Naive Bayes:
Accuracy: 0.8121269242852655
Precision: [0.92826173 0.35603715]
Recall: [0.84987369 0.55825243]
F1-score: [0.88733986 0.43478261]


So sánh thời chạy của hai mô hình

In [101]:
print(f"Logistic Regression Time: {logistic_time} seconds")
print(f"Naive Bayes Time: {nb_time} seconds")

Logistic Regression Time: 0.7288622856140137 seconds
Naive Bayes Time: 0.037023067474365234 seconds
